<a href="https://colab.research.google.com/github/eTcilopp/geek_brains/blob/main/43_data_science_libs/hw08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 8
Использование методов генерации признаков

В вашем распоряжении набор данных о качестве сна Sleep_health_and_lifestyle_dataset.csv.
Ваша задача — подготовить данные к моделированию, сгенерировать новые признаки, применить методы отбора признаков для улучшения качества предсказательной модели и подобрать оптимальные гиперпараметры для модели регрессии.

1. Предобработка данных:
— Очистите данные от пропусков и аномалий.
— Преобразуйте категориальные переменные с помощью One-Hot Encoding или Label Encoding.

2. Генерация новых признаков:
— Создайте комбинированные признаки, которые могут помочь улучшить модель.
— Используйте полиномиальные признаки для числовых переменных.

3. Отбор признаков:
— Примените несколько методов отбора признаков (например, RFE, SelectKBest).
— Сравните качество модели до и после отбора признаков.

4. Подбор гиперпараметров:
— Используйте GridSearchCV или RandomizedSearchCV для настройки параметров вашей модели регрессии.
— Оцените, как изменение гиперпараметров влияет на качество предсказаний.

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import GridSearchCV

# Загрузка и обработка данных

In [2]:
data = pd.read_csv('https://gbcdn.mrgcdn.ru/uploads/asset/6088880/attachment/90c91d3675e8310cd6c6c667b5ffd7f7.csv')

In [3]:
data.head(5)

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [4]:
data.shape

(374, 13)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


In [6]:
data.isna().sum()

,0
Person ID,0
Gender,0
Age,0
Occupation,0
Sleep Duration,0
Quality of Sleep,0
Physical Activity Level,0
Stress Level,0
BMI Category,0
Blood Pressure,0


In [7]:
data['Sleep Disorder'].unique()

array([nan, 'Sleep Apnea', 'Insomnia'], dtype=object)

In [8]:
data.fillna("Negative", inplace=True)
sleep_disorder_dict = {sleep_disorder: idx for idx, sleep_disorder in enumerate(data['Sleep Disorder'].unique())}
data['Sleep Disorder'] = data['Sleep Disorder'].replace(sleep_disorder_dict)

<ipython-input-8-eca73d6205db>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Sleep Disorder'] = data['Sleep Disorder'].replace(sleep_disorder_dict)


In [9]:
data.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,0
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,0
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,0
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,1
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,1


Обработаем тажке gender

In [10]:
data['Gender'].unique()

array(['Male', 'Female'], dtype=object)

In [11]:
data['Gender'] = data['Gender'].replace({'Male': 1, 'Female': 0})

<ipython-input-11-847013ef6f55>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Gender'] = data['Gender'].replace({'Male': 1, 'Female': 0})


In [12]:
data['Gender'] = data['Gender'].replace({'Male': 1, 'Female': 0})

In [13]:
data.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,1,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,0
1,2,1,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,0
2,3,1,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,0
3,4,1,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,1
4,5,1,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,1


Обработает "Occupation"

In [14]:
occupation_dict = {occupation: idx for idx, occupation in enumerate(data['Occupation'].unique())}

In [15]:
data['Occupation'] = data['Occupation'].replace(occupation_dict)

<ipython-input-15-c4a0a0258cda>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Occupation'] = data['Occupation'].replace(occupation_dict)


In [16]:
data.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,1,27,0,6.1,6,42,6,Overweight,126/83,77,4200,0
1,2,1,28,1,6.2,6,60,8,Normal,125/80,75,10000,0
2,3,1,28,1,6.2,6,60,8,Normal,125/80,75,10000,0
3,4,1,28,2,5.9,4,30,8,Obese,140/90,85,3000,1
4,5,1,28,2,5.9,4,30,8,Obese,140/90,85,3000,1


BMI Category

In [17]:
data['BMI Category'].unique()

array(['Overweight', 'Normal', 'Obese', 'Normal Weight'], dtype=object)

In [18]:
data['BMI Category'] = data['BMI Category'].replace({'Normal Weight': 'Normal'})

In [19]:
data['BMI Category'].value_counts()

,count
BMI Category,
Normal,216
Overweight,148
Obese,10


Приметим One-Hot Encoding преобразование

In [20]:
data['BMI Category'] = LabelEncoder().fit_transform(data['BMI Category'])

In [21]:
data.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,1,27,0,6.1,6,42,6,2,126/83,77,4200,0
1,2,1,28,1,6.2,6,60,8,0,125/80,75,10000,0
2,3,1,28,1,6.2,6,60,8,0,125/80,75,10000,0
3,4,1,28,2,5.9,4,30,8,1,140/90,85,3000,1
4,5,1,28,2,5.9,4,30,8,1,140/90,85,3000,1


Предлоложим также, что person ID не влияет на расстройство сна.

In [22]:
data.drop('Person ID', axis=1, inplace=True)

Разобъем артериальное давление на 2 колонки

In [23]:
data[['Systolic BP', 'Diastolic PB']] = data['Blood Pressure'].str.split('/', expand=True).astype(int)
data.drop('Blood Pressure', axis=1, inplace=True)

Создадим несколько синтетических признаков
<br>
Разделим продолжительность сна на уровень стресса: чем меньше стресс, тем лучше сон

In [24]:
data['Sleep_to_stress'] = data['Sleep Duration'] / data['Stress Level']

Предположим, что "нормой" будет 10000 шагов в день. Посчитаем, сколько процентов от нормы нашагивает пациент

In [25]:
data['Daily Steps Norm'] = data['Daily Steps'] / 10000

In [26]:
data

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder,Systolic BP,Diastolic PB,Sleep_to_stress,Daily Steps Norm
0,1,27,0,6.1,6,42,6,2,77,4200,0,126,83,1.016667,0.42
1,1,28,1,6.2,6,60,8,0,75,10000,0,125,80,0.775000,1.00
2,1,28,1,6.2,6,60,8,0,75,10000,0,125,80,0.775000,1.00
3,1,28,2,5.9,4,30,8,1,85,3000,1,140,90,0.737500,0.30
4,1,28,2,5.9,4,30,8,1,85,3000,1,140,90,0.737500,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,0,59,4,8.1,9,75,3,2,68,7000,1,140,95,2.700000,0.70
370,0,59,4,8.0,9,75,3,2,68,7000,1,140,95,2.666667,0.70
371,0,59,4,8.1,9,75,3,2,68,7000,1,140,95,2.700000,0.70
372,0,59,4,8.1,9,75,3,2,68,7000,1,140,95,2.700000,0.70


# Получим модель без оптимизации

In [27]:
def get_cross_val_score(method: str, model, X, y, cv=5):
    scores = cross_val_score(
        model,
        X,
        y,
        scoring=method,
        cv=5
    )
    return scores.mean()

In [28]:
X = data.drop(columns=['Sleep Disorder'])
y = data['Sleep Disorder']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [29]:
tree_model_base = DecisionTreeRegressor()

tree_model_base.fit(X_train, y_train)
y_test_predict_base = tree_model_base.predict(X_test)

In [30]:
base_r2_score_train = get_cross_val_score('r2', tree_model_base, X_train, y_train)
base_r2_score_test = get_cross_val_score('r2', tree_model_base, X_test, y_test)
base_r2_score_predict = get_cross_val_score('r2', tree_model_base, X_test, y_test_predict_base)

print(f'R2 на кросс-валидации для базовой модели на тренировочной выборке: {base_r2_score_train:.3f}')
print(f'R2 на кросс-валидации для базовой модели на тестовой выборке: {base_r2_score_test:.3f}')
print(f'R2 на кросс-валидации для базовой модели на тестовой выборке с предсказаниями: {base_r2_score_predict:.3f}')

R2 на кросс-валидации для базовой модели на тренировочной выборке: 0.596
R2 на кросс-валидации для базовой модели на тестовой выборке: 0.648
R2 на кросс-валидации для базовой модели на тестовой выборке с предсказаниями: 0.670


Результаты скорее умеренные, чем хорошие.
<br>
Попробуем выбрать признаки с помощью метода "хи-квадрат"

In [31]:
X_categorical = X.astype(int)
chi2_features = SelectKBest(chi2, k=3)
X_kbest_features = chi2_features.fit_transform(X_categorical, y)
print("Количество признаков до преобразования:", X_categorical.shape[1])
print("Количество признаков после преобразования:", X_kbest_features.shape[1])

Количество признаков до преобразования: 14
Количество признаков после преобразования: 3


In [32]:
selected_indices = chi2_features.get_support(indices=True)
feature_names = X.columns  # Если X - это DataFrame
selected_features = feature_names[selected_indices]
print("Названия выбранных признаков:", selected_features)

Названия выбранных признаков: Index(['Physical Activity Level', 'BMI Category', 'Daily Steps'], dtype='object')


Построим модель только с этими признаками

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_kbest_features, y, test_size=0.3, random_state=42)

tree_model_optim_1 = DecisionTreeRegressor()

tree_model_optim_1.fit(X_train, y_train)
y_test_predict_base = tree_model_optim_1.predict(X_test)

In [34]:
optim_r2_score_train = get_cross_val_score('r2', tree_model_optim_1, X_train, y_train)
optim_r2_score_test = get_cross_val_score('r2', tree_model_optim_1, X_test, y_test)
optim_r2_score_predict = get_cross_val_score('r2', tree_model_optim_1, X_test, y_test_predict_base)

print(f'R2 на кросс-валидации для модели с оптимизированными признаками на тренировочной выборке: {optim_r2_score_train:.3f}')
print(f'R2 на кросс-валидации для модели с оптимизированными признаками на тестовой выборке: {optim_r2_score_test:.3f}')
print(f'R2 на кросс-валидации для модели с оптимизированными признаками на тестовой выборке с предсказаниями: {optim_r2_score_predict:.3f}')

R2 на кросс-валидации для модели с оптимизированными признаками на тренировочной выборке: 0.573
R2 на кросс-валидации для модели с оптимизированными признаками на тестовой выборке: 0.608
R2 на кросс-валидации для модели с оптимизированными признаками на тестовой выборке с предсказаниями: 0.723


Качетво модели улучшилось на тренировочной и тестовой выборке, но чуть ухудшилось при предсказаниях на тестовой выборке.<br>
Зато существенно уменьшилось количество признаков!
<br>
Попробуем другой метод отбора признаков

In [35]:
select_k_best = SelectKBest(score_func=f_classif, k=5)
select_k_best.fit(X, y)
scores = select_k_best.scores_
selected_features3 = pd.DataFrame({
    'Feature': X.columns,
    'Score': scores
})
print(selected_features3.sort_values(by='Score', ascending=False))

                    Feature       Score
7              BMI Category  308.683826
11             Diastolic PB  268.097906
10              Systolic BP  214.585414
1                       Age   58.408963
5   Physical Activity Level   44.150579
8                Heart Rate   32.948639
3            Sleep Duration   31.913468
0                    Gender   31.510655
4          Quality of Sleep   27.600604
9               Daily Steps   24.776923
13         Daily Steps Norm   24.776923
12          Sleep_to_stress   11.235029
2                Occupation   10.029739
6              Stress Level    6.603565


In [36]:
selected_features = selected_features3.sort_values(by='Score', ascending=False)[:5]
selected_features

,Feature,Score
7,BMI Category,308.683826
11,Diastolic PB,268.097906
10,Systolic BP,214.585414
1,Age,58.408963
5,Physical Activity Level,44.150579


In [37]:
X_selected_features_2 = data[selected_features['Feature']]

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X_selected_features_2, y, test_size=0.3, random_state=42)

tree_model_optim_2 = DecisionTreeRegressor()

tree_model_optim_2.fit(X_train, y_train)
y_test_predict_optim_2 = tree_model_optim_2.predict(X_test)

In [39]:
optim_r2_score_train = get_cross_val_score('r2', tree_model_optim_2, X_train, y_train)
optim_r2_score_test = get_cross_val_score('r2', tree_model_optim_2, X_test, y_test)
optim_r2_score_predict = get_cross_val_score('r2', tree_model_optim_2, X_test, y_test_predict_base)

print(f'R2 на кросс-валидации для модели с оптимизированными признаками на тренировочной выборке: {optim_r2_score_train:.3f}')
print(f'R2 на кросс-валидации для модели с оптимизированными признаками на тестовой выборке: {optim_r2_score_test:.3f}')
print(f'R2 на кросс-валидации для модели с оптимизированными признаками на тестовой выборке с предсказаниями: {optim_r2_score_predict:.3f}')

R2 на кросс-валидации для модели с оптимизированными признаками на тренировочной выборке: 0.617
R2 на кросс-валидации для модели с оптимизированными признаками на тестовой выборке: 0.555
R2 на кросс-валидации для модели с оптимизированными признаками на тестовой выборке с предсказаниями: 0.769


Качество модели немного улучшилось и равномерно для всех комбинаций данных

# Подбор гиперпараметров

In [40]:
# Поиск гиперпараметров с помощью сетки параметров
param_grid = {
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 4, 6]
}

grid_search = GridSearchCV(
    estimator=tree_model_optim_2,
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    n_jobs=-1
)
grid_search.fit(X_train, y_train)

print(f"Лучшие гиперпараметры для модели из коробки (Grid Search): {grid_search.best_params_}")
print(f"Лучшая оценка для модели из коробки (Grid Search): {grid_search.best_score_:.2f}")

Лучшие гиперпараметры для модели из коробки (Grid Search): {'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 2}
Лучшая оценка для модели из коробки (Grid Search): 0.65


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Посмотрим на результаты работы модели с этими параметрами

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X_selected_features_2, y, test_size=0.3, random_state=42)

tree_model_optim_2 = DecisionTreeRegressor(**grid_search.best_params_)

tree_model_optim_2.fit(X_train, y_train)
y_test_predict_optim_2 = tree_model_optim_2.predict(X_test)

In [42]:
optim_r2_score_train = get_cross_val_score('r2', tree_model_optim_2, X_train, y_train)
optim_r2_score_test = get_cross_val_score('r2', tree_model_optim_2, X_test, y_test)
optim_r2_score_predict = get_cross_val_score('r2', tree_model_optim_2, X_test, y_test_predict_base)

print(f'R2 на кросс-валидации для модели с оптимизированными признаками на тренировочной выборке: {optim_r2_score_train:.3f}')
print(f'R2 на кросс-валидации для модели с оптимизированными признаками на тестовой выборке: {optim_r2_score_test:.3f}')
print(f'R2 на кросс-валидации для модели с оптимизированными признаками на тестовой выборке с предсказаниями: {optim_r2_score_predict:.3f}')

R2 на кросс-валидации для модели с оптимизированными признаками на тренировочной выборке: 0.645
R2 на кросс-валидации для модели с оптимизированными признаками на тестовой выборке: 0.696
R2 на кросс-валидации для модели с оптимизированными признаками на тестовой выборке с предсказаниями: 0.690


Модель демонстрирует низкое смещение (bias), так как разница между тренировочной и тестовой выборкой минимальна.
<br>
Низкая вариативность (variance), так как результаты на тестовой выборке и на предсказаниях совпадают.
<br>
Следовательно, можно утверждать, что у модели хороший баланс между bias и variance, что говорит о её стабильности и адекватной общей производительности.

Попробуем другой метод подбора параметров

In [45]:
!pip install scikit-optimize -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.4 MB/s eta 0:00:00


In [50]:
from skopt import BayesSearchCV
from skopt.space import Integer, Real

param_space = {
    'max_depth': Integer(3, 30),               # Максимальная глубина дерева
    'min_samples_split': Integer(2, 10),      # Минимальное число образцов для разделения узла
    'min_samples_leaf': Integer(1, 5),        # Минимальное число образцов в листе
    'max_features': Real(0.1, 1.0, prior='uniform')  # Доля признаков для рассмотрения
}

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X_selected_features_2, y, test_size=0.3, random_state=42)

In [52]:
from sklearn.ensemble import RandomForestClassifier

model = DecisionTreeRegressor()

bayes_search = BayesSearchCV(
    model,
    param_space,
    n_iter=50,  # Число итераций поиска
    cv=5        # Кросс-валидация на 5 фолдов
)
bayes_search.fit(X_train, y_train)

# Вывод наилучших гиперпараметров и оценки
print("Best Hyperparameters:", bayes_search.best_params_)
print("Best Cross-Validation Score:", bayes_search.best_score_)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Hyperparameters: OrderedDict([('max_depth', 20), ('max_features', 0.5101762889653508), ('min_samples_leaf', 5), ('min_samples_split', 4)])
Best Cross-Validation Score: 0.7062783435698403


Применим найденные параметры к модели

In [53]:
tree_model_optim_2 = DecisionTreeRegressor(**bayes_search.best_params_)

tree_model_optim_2.fit(X_train, y_train)
y_test_predict_optim_2 = tree_model_optim_2.predict(X_test)

In [54]:
optim_r2_score_train = get_cross_val_score('r2', tree_model_optim_2, X_train, y_train)
optim_r2_score_test = get_cross_val_score('r2', tree_model_optim_2, X_test, y_test)
optim_r2_score_predict = get_cross_val_score('r2', tree_model_optim_2, X_test, y_test_predict_base)

print(f'R2 на кросс-валидации для модели с оптимизированными признаками на тренировочной выборке: {optim_r2_score_train:.3f}')
print(f'R2 на кросс-валидации для модели с оптимизированными признаками на тестовой выборке: {optim_r2_score_test:.3f}')
print(f'R2 на кросс-валидации для модели с оптимизированными признаками на тестовой выборке с предсказаниями: {optim_r2_score_predict:.3f}')

R2 на кросс-валидации для модели с оптимизированными признаками на тренировочной выборке: 0.617
R2 на кросс-валидации для модели с оптимизированными признаками на тестовой выборке: 0.627
R2 на кросс-валидации для модели с оптимизированными признаками на тестовой выборке с предсказаниями: 0.745


Результаты улучшились!<br>

# Выводы:
Оптимизация модели методом оптимизации набора входных параметров (фич)<br> и методом оптимизации значений гиперпараметров является сложной и творческой задачей.
<br>
Грамотный подбор фич и значений гиперпараметров улучшает модель и позволяет получить лучшие результаты